## Directions to run 
outside the docker 
env: sg3 

create a directory for new experiment

In [1]:
import os 

# set environment variables to make only 1 GPU visible, and select it
# os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
# os.environ["CUDA_VISIBLE_DEVICES"]="1"

# check if GPU is visible
import torch
torch.cuda.device_count()


1

In [2]:
DRAGVIDEO_ROOT_PATH = "/Ext_4T_SSD/ASHOK/"

In [3]:
lazy_config = {
    "EXP_NAME": "exp_01_pti_1_inv_1",
    "e4e": f"{DRAGVIDEO_ROOT_PATH}DragVideo/DragGAN/_PTI/pretrained_models/e4e_ffhq_encode.pt",
    "stylegan2_ada_ffhq": f"{DRAGVIDEO_ROOT_PATH}DragVideo/DragGAN/_PTI/pretrained_models/ffhq.pkl",
    "video_path":  f"{DRAGVIDEO_ROOT_PATH}DragVideo/Data_store/OLD/original_videos/person_speaking_original.mp4",
    "model_name" : "stylegan2",
    "n_frames" : 1,
    "IMAGE_SIZE": 1024,
    "N_STEPS_in_draggan": "1",
    

}

env_config = {
    # "DragGAN_dir": f"{DRAGVIDEO_ROOT_PATH}DragVideo/DragGAN",
    # "Experiment_base_path":f"{DRAGVIDEO_ROOT_PATH}DragVideo/Data_store/experiments/' ,
    # "dummy_config_path" : f"{DRAGVIDEO_ROOT_PATH}DragVideo/DragGAN/_PTI/configs/dummy',

    
    # for outside the container
    "DragGAN_dir": "/Ext_4T_SSD/ASHOK/DragVideo/DragGAN",
    "Experiment_base_path":'/Ext_4T_SSD/ASHOK/DragVideo/Data_store/experiments/' ,
    "init_exp_dir_shell_path": "/Ext_4T_SSD/ASHOK/DragVideo/DragGAN/_PTI/init_datadirs.sh",
    "dummy_config_path" : '/Ext_4T_SSD/ASHOK/DragVideo/DragGAN/_PTI/configs/dummy',

}

#hyper parameters in PTI
hyper_config = {
    "max_pti_steps": 1,#500,
    "first_inv_steps": 1,#00,
    "max_images_to_invert": 200,
}

In [4]:
from run_utils import *

# create experiment data folder structure
Experiment_name = lazy_config["EXP_NAME"]
Experiment_base_path = env_config["Experiment_base_path"]
Experiment_path = os.path.join(Experiment_base_path, Experiment_name)

init_experiment_dir(Experiment_name,Experiment_base_path,shell_script_path=env_config["init_exp_dir_shell_path"])

Experiment directory already exists


set paths for e4e, stylegan used in PTI

In [5]:
# dummy paths_config overwrites the paths_config.py
# dummy_config_path = f"{DRAGVIDEO_ROOT_PATH}DragVideo/DragGAN/_PTI/configs/dummy'
add_dummy_config_from_dict("hyperparameters.py", hyper_config,ROOT_PATH=env_config["dummy_config_path"])

In [6]:
# change path configs , hyperparameters 
paths_config_dict = {
    #pretrained models
    "e4e": lazy_config["e4e"],
    "stylegan2_ada_ffhq": lazy_config["stylegan2_ada_ffhq"],
    
    # to store tuned stylegan weights
    "checkpoints_dir": os.path.join(Experiment_path,'tuned_SG'),
    # to store latents
    "embedding_base_dir": os.path.join(Experiment_path,'latents'),
    # aligned / processed images
    "input_data_path": os.path.join(Experiment_path,'aligned'),
}

add_dummy_config_from_dict("paths_config.py", paths_config_dict,ROOT_PATH=env_config["dummy_config_path"])

In [7]:
from importlib.machinery import SourceFileLoader
# imports the module from the given path
video_utils = SourceFileLoader("video_utils","../utils_draggan/video_utils.py").load_module()
raw_path = os.path.join(Experiment_path, "raw")
from utils.align_data import pre_process_images

video_utils.extract_frames(lazy_config['video_path'], raw_path,n_frames=lazy_config['n_frames'])
pre_process_images(raw_path, IMAGE_SIZE=lazy_config['IMAGE_SIZE']) # o/p: config.input_data_path 

Creating.../Ext_4T_SSD/ASHOK/DragVideo/Data_store/experiments/exp_01_pti_1_inv_1/raw/000.jpg


100%|██████████| 1/1 [00:00<00:00,  1.08it/s]


In [8]:

from utils.align_data import pre_process_images
from scripts.run_pti import run_PTI

from run_utils_2 import load_generators,export_updated_pickle
from configs import paths_config

use_multi_id_training = True
model_id = run_PTI(use_wandb=False, use_multi_id_training=use_multi_id_training)


Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /Ext_4T_SSD/ASHOK/DragVideo/env/sg3/lib/python3.9/site-packages/lpips/weights/v0.1/alex.pth
current_directory:  /Ext_4T_SSD/ASHOK/DragVideo/DragGAN/_PTI
current_directory:  /Ext_4T_SSD/ASHOK/DragVideo/DragGAN/_PTI
torch.Size([3, 1024, 1024]) 3 1024 1024
Setting up PyTorch plugin "bias_act_plugin"... Done.


  0%|          | 0/1 [00:00<?, ?it/s]

Setting up PyTorch plugin "upfirdn2d_plugin"... 

100%|██████████| 1/1 [00:00<00:00,  1.01it/s]


Done.


100%|██████████| 1/1 [00:00<00:00,  7.07it/s]


In [9]:
use_multi_id_training = True

generator_type =paths_config.multi_id_model_type if use_multi_id_training else image_name
old_G, new_G = load_generators(model_id, generator_type)
sg_tuned_pkl = export_updated_pickle(new_G,model_id,name = lazy_config["model_name"])

print(sg_tuned_pkl) # 'QBUXQCXZGWET'

Exporting large updated pickle based off new generator and ffhq.pkl
/Ext_4T_SSD/ASHOK/DragVideo/Data_store/experiments/exp_01_pti_1_inv_1/tuned_SG/stylegan2_GIFNUZUFBIPS.pkl


## landmarks

In [10]:
#get landmarks
landmark_path = f"{DRAGVIDEO_ROOT_PATH}DragVideo/DragGAN/_facial-landmarks-recognition"
os.chdir(landmark_path)

from main import landmarks, dict_landmarks,show_landmarks,get_landmarks_dir

# to store landmarks
landmarks_dir =  os.path.join(Experiment_path,'landmarks')
processed_images_dir =  os.path.join(Experiment_path,'aligned')

# generate landmarks for all images in processed_images_dir
get_landmarks_dir(processed_images_dir,landmarks_dir)

landmark_path:  /Ext_4T_SSD/ASHOK/DragVideo/Data_store/experiments/exp_01_pti_1_inv_1/landmarks/000.pkl


In [11]:
os.chdir(env_config["DragGAN_dir"])
import subprocess
subprocess.call(['python', 'run_dragvideo.py', '--Experiment_path', Experiment_path, '--N_STEPS', lazy_config["N_STEPS_in_draggan"], '--CHECKPOINT_PATH', sg_tuned_pkl])

intiating global state....
calling init_images......
Loading "/Ext_4T_SSD/ASHOK/DragVideo/Data_store/experiments/exp_01_pti_1_inv_1/tuned_SG/stylegan2_GIFNUZUFBIPS.pkl"... Done.
()
{'z_dim': 512, 'c_dim': 0, 'w_dim': 512, 'img_resolution': 1024, 'img_channels': 3, 'mapping_kwargs': {'num_layers': 8, 'embed_features': None, 'layer_features': None, 'activation': 'lrelu', 'lr_multiplier': 0.01, 'w_avg_beta': 0.995}, 'synthesis_kwargs': {'channel_base': 32768, 'channel_max': 512, 'num_fp16_res': 4, 'conv_clamp': 256, 'architecture': 'skip', 'resample_filter': [1, 3, 3, 1], 'use_noise': True, 'activation': 'lrelu'}}
Setting up PyTorch plugin "bias_act_plugin"... Done.
Setting up PyTorch plugin "upfirdn2d_plugin"... Done.
Running with:
    Source: [array([258, 489]), array([262, 566]), array([269, 642]), array([277, 720]), array([301, 793]), array([339, 861]), array([383, 927]), array([437, 986]), array([ 507, 1003]), array([580, 988]), array([641, 936]), array([694, 876]), array([740, 811])

/Ext_4T_SSD/ASHOK/DragVideo/env/sg3/lib/python3.9/site-packages/torch/nn/functional.py:3609: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  warnings.warn(


Current Source:
    [258, 489]
    [262, 566]
    [269, 642]
    [277, 720]
    [301, 793]
    [339, 861]
    [383, 927]
    [437, 986]
    [507, 1003]
    [580, 988]
    [641, 936]
    [694, 876]
    [740, 811]
    [770, 739]
    [782, 659]
    [793, 578]
    [804, 496]
    [290, 448]
    [311, 401]
    [359, 375]
    [413, 373]
    [464, 389]
    [554, 382]
    [609, 368]
    [666, 373]
    [715, 403]
    [742, 450]
    [506, 465]
    [503, 525]
    [500, 583]
    [498, 643]
    [441, 668]
    [468, 683]
    [500, 696]
    [536, 682]
    [568, 669]
    [357, 488]
    [383, 477]
    [412, 477]
    [442, 484]
    [413, 486]
    [385, 488]
    [582, 484]
    [610, 476]
    [639, 478]
    [666, 488]
    [637, 488]
    [610, 486]
    [399, 766]
    [432, 754]
    [471, 749]
    [504, 758]
    [538, 750]
    [582, 755]
    [624, 768]
    [587, 831]
    [542, 856]
    [505, 860]
    [469, 855]
    [429, 829]
    [415, 770]
    [471, 768]
    [504, 771]
    [538, 768]
    [608, 772]
    [541

0

# OLD


In [12]:
ala

NameError: name 'ala' is not defined

In [ ]:
# https://github.com/danielroich/PTI

# run below commands from /PTI/ 
# export PYTHONPATH=$PYTHONPATH:$PWD

# change path inside it.
# python utils/align_data.py

# path_ = f"{DRAGVIDEO_ROOT_PATH}DragVideo/PTI"
# import os 
# os.chdir(path_)


In [ ]:
# import os 
# dir = "/workspace/PTI/data/raw_images"
# # it has images from 0 to 400 .jpeg 
# # rename them so that all are 3 digits
# for i in range(400):
#     if i < 10:
#         os.rename(dir + "/" + str(i) + ".jpg", dir + "/00" + str(i) + ".jpg")
#     elif i < 100:
#         os.rename(dir + "/" + str(i) + ".jpg", dir + "/0" + str(i) + ".jpg")
#     else:
#         continue
#         # os.rename(dir + "/" + str(i) + ".jpeg", dir + "/" + str(i) + ".jpeg")


In [ ]:
import os 
import sys
root_dir = f"{DRAGVIDEO_ROOT_PATH}DragVideo'
os.chdir(f'{root_dir}/PTI')

use_multi_id_training = True

from scripts.run_pti import run_PTI
model_id = run_PTI(use_wandb=False, use_multi_id_training=use_multi_id_training)

FileNotFoundError: [Errno 2] No such file or directory: '/home/bean/DragVideo/PTI'

In [ ]:
import os
import sys
import pickle
import numpy as np
from PIL import Image
import torch
from configs import paths_config, hyperparameters, global_config
from utils.align_data import pre_process_images
from scripts.run_pti import run_PTI
from IPython.display import display
import matplotlib.pyplot as plt
from scripts.latent_editor_wrapper import LatentEditorWrapper

In [ ]:
def load_generators(model_id, image_name):
  with open(paths_config.stylegan2_ada_ffhq, 'rb') as f:
    old_G = pickle.load(f)['G_ema'].cuda()

  with open(f'{paths_config.checkpoints_dir}/model_{model_id}_{image_name}.pt', 'rb') as f_new:
    new_G = torch.load(f_new).cuda()

  return old_G, new_G

In [ ]:
# image_name = "000"

# # /home/bean/DragVideo/DragGAN/PTI_results/checkpoints/model_ZEWLQSQSSQWA_000.pt
# model_id = "ZEWLQSQSSQWA"

In [ ]:
model_id = f"{DRAGVIDEO_ROOT_PATH}DragVideo/Data_store/model_weights/sg3_3rdtime_weights/sg3-r-ffhq-1024_module.pt'
use_multi_id_training = True

generator_type = paths_config.multi_id_model_type if use_multi_id_training else image_name
old_G, new_G = load_generators(model_id, generator_type)

In [ ]:
#code from : https://github.com/danielroich/PTI/issues/26 , plus little bit modification

def export_updated_pickle(new_G,model_id):
  print("Exporting large updated pickle based off new generator and ffhq.pkl")
  with open(paths_config.stylegan2_ada_ffhq, 'rb') as f:
    d = pickle.load(f)
    old_G = d['G_ema'].cuda()
    old_D = d['D'].eval().requires_grad_(False).cpu()

  tmp = {}
  tmp['G'] = old_G.eval().requires_grad_(False).cpu()
  tmp['G_ema'] = new_G.eval().requires_grad_(False).cpu()
  tmp['D'] = old_D
  tmp['training_set_kwargs'] = None
  tmp['augment_pipe'] = None

  # with open(f'{paths_config.checkpoints_dir}/stylegan3_{model_id}.pkl', 'wb') as f:
  #     pickle.dump(tmp, f)

export_updated_pickle(new_G,model_id)

In [ ]:
model_id # 'AXOCWYGCAEDY'for 10 images

'ZEWLQSQSSQWA'

In [ ]:
# # .pt to .pkl file conversion

# sg3_pt_file_path = f"{DRAGVIDEO_ROOT_PATH}DragVideo/Data_store/model_weights/sg3_3rdtime_weights/sg3-r-ffhqu-1024.pt' #downloaded
# # sg3_pt_file_path_full=f"{DRAGVIDEO_ROOT_PATH}DragVideo/Data_store/data/PTI_results/checkpoints/model_HCQDSNSTNNJM_multi_id.pt"

# with open(sg3_pt_file_path, 'rb') as f_new:
#     weights = torch.load(f_new)#.cuda()

# # with open(sg3_pt_file_path_full, 'rb') as f_new:
# #     module = torch.load(f_new)#.cuda()

In [ ]:
# sg3_1024_pkl = f"{DRAGVIDEO_ROOT_PATH}DragVideo/PTI/pretrained_models/stylegan3-r-ffhq-1024x1024.pkl"
# with open(sg3_1024_pkl,'rb') as f:
#     sg3_1024 = pickle.load(f)

# torch.save(sg3_1024["G"],f"{DRAGVIDEO_ROOT_PATH}DragVideo/Data_store/model_weights/sg3_3rdtime_weights/sg3-r-ffhqu-1024_module.pt')

In [ ]:
def export_updated_pickle(new_G,model_id):
  print("Exporting large updated pickle based off new generator and ffhq.pkl")
  with open(paths_config.stylegan2_ada_ffhq, 'rb') as f:
    d = pickle.load(f)
    old_G = d['G_ema'].cuda()
    old_D = d['D'].eval().requires_grad_(False).cpu()

  tmp = {}
  tmp['G'] = old_G.eval().requires_grad_(False).cpu()
  tmp['G_ema'] = new_G.eval().requires_grad_(False).cpu()
  tmp['D'] = old_D
  tmp['training_set_kwargs'] = None
  tmp['augment_pipe'] = None

  with open(f'{paths_config.checkpoints_dir}/stylegan2_{model_id}.pkl', 'wb') as f:
      pickle.dump(tmp, f)

model_id = '3rdtime_test' 
export_updated_pickle(new_G,model_id)

In [ ]:
sg3_3rdtime_pt = f"{DRAGVIDEO_ROOT_PATH}DragVideo/Data_store/model_weights/sg3_3rdtime_weights/sg3-r-ffhq-1024_module.pt'
use_multi_id_training = True
sg3_1024_pkl = f"{DRAGVIDEO_ROOT_PATH}DragVideo/PTI/pretrained_models/stylegan3-r-ffhq-1024x1024.pkl"



In [ ]:
with open(sg3_1024_pkl, 'rb') as f:
    old_G = pickle.load(f)['G_ema'].cuda()

with open(sg3_3rdtime_pt, 'rb') as f_new:
    new_G = torch.load(f_new).cuda()

In [ ]:
with open(sg3_1024_pkl, 'rb') as f:
    d = pickle.load(f)
    old_G = d['G_ema'].cuda()
    old_D = d['D'].eval().requires_grad_(False).cpu()

tmp = {}
tmp['G'] = old_G.eval().requires_grad_(False).cpu()
tmp['G_ema'] = new_G.eval().requires_grad_(False).cpu()
tmp['D'] = old_D
tmp['training_set_kwargs'] = None
tmp['augment_pipe'] = None

with open(f"{DRAGVIDEO_ROOT_PATH}DragVideo/Data_store/model_weights/sg3_3rdtime_weights/sg3-r-ffhq-1024_module.pkl', 'wb') as f:
    pickle.dump(tmp, f)

In [ ]:
sg3u_3rdtime_pt = f"{DRAGVIDEO_ROOT_PATH}DragVideo/Data_store/model_weights/sg3_3rdtime_weights/sg3-r-ffhqu-1024_module.pt'
use_multi_id_training = True
sg3_1024_pkl = f"{DRAGVIDEO_ROOT_PATH}DragVideo/PTI/pretrained_models/stylegan3-r-ffhq-1024x1024.pkl"

In [ ]:
with open(sg3_1024_pkl, 'rb') as f:
    old_G = pickle.load(f)['G_ema'].cuda()

with open(sg3u_3rdtime_pt, 'rb') as f_new:
    new_G = torch.load(f_new).cuda()

In [ ]:
with open(sg3_1024_pkl, 'rb') as f:
    d = pickle.load(f)
    old_G = d['G_ema'].cuda()
    old_D = d['D'].eval().requires_grad_(False).cpu()

tmp = {}
tmp['G'] = old_G.eval().requires_grad_(False).cpu()
tmp['G_ema'] = new_G.eval().requires_grad_(False).cpu()
tmp['D'] = old_D
tmp['training_set_kwargs'] = None
tmp['augment_pipe'] = None

with open(f"{DRAGVIDEO_ROOT_PATH}DragVideo/Data_store/model_weights/sg3_3rdtime_weights/sg3-r-ffhqu-1024_module.pkl', 'wb') as f:
    pickle.dump(tmp, f)